# DLC Live PyTorch Demo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dlclive import DLCLive, Processor
from dlclive.display import Display
import cv2
import numpy as np
from pathlib import Path
import time

Loading DLC 3.0.0rc4...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


/home/dikra/miniconda3/envs/dlc-live/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Snapshot to ONNX model 

In [12]:
# In case you do not have a .onnx model exported, use this cell to export your DLC3.0 snapshot

from deeplabcut.pose_estimation_pytorch.config import read_config_as_dict
from deeplabcut.pose_estimation_pytorch.models import PoseModel
import torch
import onnxruntime as ort

device = "cuda" if torch.cuda.is_available() else "cpu"

root = Path("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait")
model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
weights_path = root / "snapshot-200.pt"

model = PoseModel.build(model_cfg["model"])
weights = torch.load(weights_path, map_location=device)
model.load_state_dict(weights["model"])

dummy_input = torch.zeros((1, 3, 224, 224))

torch.onnx.export(
    model,
    dummy_input,
    "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait/resnet.onnx",
    verbose=False,
    input_names=["input"],
    dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"}},
)

/tmp/ipykernel_128437/141163355.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(weights_path, map_location=device)


In [25]:
# Load test frame
img = cv2.imread("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait/img008.png")

### DLC Live with ONNX exported DLC 3.0 model

In [17]:
# Dikra
onnx_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait",
    model_type="onnx",
    device="cuda",
    display=True,
)

# Anna
# onnx_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# onnx_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

onnx_pose = onnx_dlc_live.init_inference(frame=img)
onnx_pose

Loading the model took 0.20421051979064941 sec
ONNX inference took 1.2931642532348633 sec


{'poses': tensor([[[[3.6219e+02, 4.1237e+02, 5.0485e-01],
           [3.6271e+02, 3.9583e+02, 5.8573e-01],
           [3.5676e+02, 3.7446e+02, 7.0502e-01],
           [3.4043e+02, 3.5935e+02, 7.1149e-01],
           [3.3437e+02, 4.0497e+02, 6.5126e-01],
           [3.3771e+02, 4.0689e+02, 4.4916e-01],
           [2.0013e+02, 4.3546e+02, 5.2069e-01],
           [3.4131e+02, 4.3089e+02, 3.4870e-01],
           [2.1540e+02, 3.9426e+02, 7.8363e-01],
           [1.9974e+02, 3.7728e+02, 5.2781e-01],
           [1.8255e+02, 3.6741e+02, 6.4685e-01],
           [2.7519e+02, 3.3175e+02, 5.7629e-01],
           [3.5014e+02, 4.3089e+02, 4.1489e-01],
           [4.3459e+02, 3.1565e+02, 8.1676e-01],
           [4.2561e+02, 2.9912e+02, 8.8822e-01],
           [2.4359e+02, 5.0565e+02, 5.7643e-01],
           [3.5792e+02, 4.8275e+02, 3.0069e-01],
           [4.0234e+02, 4.1449e+02, 5.4244e-01],
           [1.7143e+02, 4.8666e+02, 4.5818e-01],
           [3.2110e+02, 5.1144e+02, 2.5129e-01],
           

![Screenshot from 2024-08-20 14-29-53.png](./docs/assets/Screenshot%20from%202024-08-20%2014-36-00.png)

In [18]:
onnx_pose = onnx_dlc_live.get_pose(frame=img)
onnx_pose

ONNX inference took 0.04568004608154297 sec


{'poses': tensor([[[[3.6219e+02, 4.1237e+02, 5.0485e-01],
           [3.6271e+02, 3.9583e+02, 5.8573e-01],
           [3.5676e+02, 3.7446e+02, 7.0502e-01],
           [3.4043e+02, 3.5935e+02, 7.1149e-01],
           [3.3437e+02, 4.0497e+02, 6.5126e-01],
           [3.3771e+02, 4.0689e+02, 4.4916e-01],
           [2.0013e+02, 4.3546e+02, 5.2069e-01],
           [3.4131e+02, 4.3089e+02, 3.4870e-01],
           [2.1540e+02, 3.9426e+02, 7.8363e-01],
           [1.9974e+02, 3.7728e+02, 5.2781e-01],
           [1.8255e+02, 3.6741e+02, 6.4685e-01],
           [2.7519e+02, 3.3175e+02, 5.7629e-01],
           [3.5014e+02, 4.3089e+02, 4.1489e-01],
           [4.3459e+02, 3.1565e+02, 8.1676e-01],
           [4.2561e+02, 2.9912e+02, 8.8822e-01],
           [2.4359e+02, 5.0565e+02, 5.7643e-01],
           [3.5792e+02, 4.8275e+02, 3.0069e-01],
           [4.0234e+02, 4.1449e+02, 5.4244e-01],
           [1.7143e+02, 4.8666e+02, 4.5818e-01],
           [3.2110e+02, 5.1144e+02, 2.5129e-01],
           

In [19]:
onnx_dlc_live.display.destroy()

### DLC Live with snaptshot of DLC 3.0 model (.pt)

In [22]:
# Dikra
pytorch_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch",
    display=True,
)

# Anna
# pytorch_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# pytorch_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

pytorch_pose = pytorch_dlc_live.init_inference(frame=img)
pytorch_pose

NameError: name 'img' is not defined

In [22]:
pytorch_dlc_live.display.destroy()

![PyTorch model inference](./docs/assets/Screenshot%20from%202024-08-20%2014-29-53.png)

### Which is faster?

In [3]:
import glob
import os

root = "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait"
test_images = glob.glob(os.path.normpath(root + "/*.png"))


def mean_time_inference(dlc_live, images):
    times = []
    for i, img_p in enumerate(images):
        img = cv2.imread(img_p)

        if i == 0:
            start = time.time()
            dlc_live.init_inference(img)
            end = time.time()
        else:
            start = time.time()
            dlc_live.get_pose(img)
            end = time.time()
        times.append(end - start)
    print(times)

    return np.mean(times)

In [5]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait",
    device="cuda",
    model_type="onnx",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"TOTAL Inference of ONNX model took on average {mean_time} seconds for {len(test_images)} images"
)

Loading the model took 0.4884378910064697 sec
ONNX inference took 2.5211031436920166 sec
ONNX postprocessing took 0.0028717517852783203 sec
[3.2065136432647705]
TOTAL Inference of ONNX model took on average 3.2065136432647705 seconds for 1 images


In [19]:
dlc_live.display.destroy()

In [7]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait",
    snapshot="snapshot-263.pt",
    device="cuda",
    model_type="pytorch",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"Inference of PyTorch model took on average {mean_time} seconds for {len(test_images)} images"
)

/home/dikra/MyHub/Code/DLC24_Hub/DLC_AI2024/DeepLabCut-live/dlclive/dlclive.py:257: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(model_path, map_locati

Loading the model took 2.3344688415527344 sec
PyTorch inference took 0.2182178497314453 sec
PyTorch postprocessing took 0.06518125534057617 sec
[2.751272201538086]
Inference of PyTorch model took on average 2.751272201538086 seconds for 1 images


In [8]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/ventral-gait",
    device="tensorrt",
    model_type="onnx",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"Inference of PyTorch model took on average {mean_time} seconds for {len(test_images)} images"
)

Loading the model took 4.5614917278289795 sec
ONNX inference took 47.8155951499939 sec
ONNX postprocessing took 0.0016188621520996094 sec
[52.50954818725586]
Inference of PyTorch model took on average 52.50954818725586 seconds for 1 images


In [10]:
dlc_live.sess.get_providers()

['TensorrtExecutionProvider', 'CPUExecutionProvider']

In [9]:
dlc_live.display.destroy()